# Exploring Autoencoders Applied to Neural Recordings

In [1]:
import os
import random
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('paper', font_scale=2.0)
sns.set_style('ticks')
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import Input, Model, regularizers
from keras.layers import Dense
from keras.utils import to_categorical
from knn import test_knn

SIZES = []
RED_DIM = 3

In [2]:
### data setup

path = os.getcwd() # put this file in the same dir as the root dir of the training data
data_root = os.path.join(path,'tactile-coding')
subj_paths = [d for d in os.listdir(data_root) if int(d) <= 12] # only first 12 subjs have EEG data

dataset = []
for i, subj in enumerate(subj_paths):
    subj_path = os.path.join(data_root,subj,'tables')
    
    # already sorted based on source id --> rows correspond in each
    waveforms = pd.read_csv(os.path.join(subj_path,'waveforms.csv'))
    units = pd.read_csv(os.path.join(subj_path,'units.csv'))
    cell_types = units['cellType']
    
    # alignment
    waveforms['sourceId'] = units['sourceId']
    waveforms.set_index('sourceId',inplace = True)
    waveforms.columns = waveforms.columns.astype('float')
    
    waveforms['subj'] = i
    data = pd.concat([waveforms,cell_types],axis=1)
    dataset += [data]

dataset = pd.concat(dataset)

In [3]:
### extraction and preprocessing

X_df = dataset.drop(['cellType','subj'],axis=1)
G_df = dataset['cellType']
features = X_df.columns.values

# feature scaling to standard normal
def standardize(arr):
    return (arr - np.mean(arr,axis=1).reshape(-1,1)) / np.std(arr,axis=1).reshape(-1,1)

def normalize(arr):
    return (arr - arr.min(axis=1).reshape(-1,1)) / (arr.max(axis=1).reshape(-1,1) - arr.min(axis=1).reshape(-1,1))

X = normalize(standardize(X_df.values))
G = G_df.values

y = np.zeros(len(G))
cells = np.unique(G)
for i, cell in enumerate(cells):
    y[G==cell] = i

y = y.astype(int)
y_cat = to_categorical(y,3)

Note that in this notebook the recordings are normalized between 0 and 1. *For summary stats and preliminary waveform visualizations, see the neural_PCA notebook.* 

## Autoencoder

In [4]:
### autoencoder builder function

def create_ae(init='uniform',activation='relu',optimizer='adam',loss='mean_squared_error',ret_comp=False):
    """
    sizes: list of layer sizes in decreasing order until encoding --> mirrored for decoding
    ret_all: returns components of the autoencoder as well if True
    """
    global SIZES
    
    if len(SIZES) < 3: 
        raise ValueError("Autoencoder must have at least 3 layers.")
    
    enc_inp = Input(shape=(SIZES[0],))
    enc = enc_inp
    for size in SIZES[1:]:
        enc = Dense(size,kernel_initializer=init,activation=activation)(enc)
        
    dec_inp = Input(shape=(SIZES[-1],))
    dec = dec_inp
    for size in reversed(SIZES[:-1]):
        dec = Dense(size,kernel_initializer=init,activation=activation)(dec)
    
    encoder = Model(enc_inp,enc,name='encoder')
    decoder = Model(dec_inp,dec,name='decoder')
    autoencoder = Model(enc_inp,decoder(encoder(enc_inp)),name='autoencoder')
    
    autoencoder.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return [encoder, decoder] if ret_comp else autoencoder

In [5]:
### searching for the best hyperparams
"""
References: 
https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
https://stackoverflow.com/questions/49823192/autoencoder-gridsearch-hyperparameter-tuning-keras
https://towardsdatascience.com/autoencoders-for-the-compression-of-stock-market-data-28e8c1a2da3e
"""

SIZES = [X.shape[1],X.shape[1]//2,X.shape[1]//4,RED_DIM]
model = KerasClassifier(build_fn=create_ae,verbose=False)

batch_size = [10, 20, 30]
epochs = [10, 25, 50]
init = ['uniform', 'normal','glorot_uniform','variance_scaling']
activation = ['linear','relu','sigmoid','tanh','selu','elu']
optimizer = ['SGD', 'Adam','rmsprop']
loss = ['mean_squared_error','mean_absolute_error']
param_grid = dict(batch_size=batch_size, 
                  epochs=epochs,
                  init=init, 
                  activation=activation,
                  optimizer=optimizer,
                  loss=loss
                 )

search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=True)
search_results = search.fit(X, X)
score = search_results.best_score_
params = search_results.best_params_

print("\nSelected hyperparameters: %f using %s" % (score,params))

Fitting 3 folds for each of 1296 candidates, totalling 3888 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  7.1min
/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 3888 out of 3888 | elapsed: 11.5min finished



Selected hyperparameters: 0.666667 using {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'init': 'uniform', 'loss': 'mean_absolute_error', 'optimizer': 'SGD'}


Debugging this grid search was truly a trial in patience. Scikit has an underlying [bug in its GridSearchCV function](https://github.com/keras-team/keras/issues/13586) that may throw an error on any hyperparameter that modifies the architecture of the network. Here, this bug popped up a few times for layer tuning. So in order to tune the layers and sizes a global parameter needs to be set prior to grid searching. Overall, scikit's grid search is very buggy in tandem with the Keras wrapper; another bug reared when I tried to tune the loss function with cosine similarity. On a lighter note, normalizing the recordings to [0,1] enormously impacted the model, bringing the final grid search accuracy from <50% to 100%. As many guides note, the grid search implementation has underlying randomization so running multiple times and averaging might be a good idea; however, I've found that once a good set of params is available then the search will usually hover around those consistently upon repeat. 

In [6]:
### fit model with selected params

encoder, decoder = create_ae(params['init'],params['activation'],params['optimizer'],ret_comp=True)

inp = Input(shape=(SIZES[0],))
ae = Model(inp,decoder(encoder(inp)),name='ae')
ae.compile(optimizer=params['optimizer'], loss=params['loss'], metrics=['accuracy'])

X_tr, X_te = train_test_split(X,test_size=0.1,random_state=42)
X_tr, X_val = train_test_split(X_tr,test_size=0.1,random_state=42)
history = ae.fit(X_tr, X_tr,
                epochs=params['epochs'],
                batch_size=params['batch_size'],
                shuffle=True,
                validation_data=(X_val, X_val))

Epoch 1/10
25/25 [==============================] - 0s 5ms/step - loss: 0.7205 - accuracy: 0.0041 - val_loss: 0.7216 - val_accuracy: 0.0000e+00
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 0.7172 - accuracy: 0.0000e+00 - val_loss: 0.7183 - val_accuracy: 0.0000e+00
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 0.7139 - accuracy: 0.0000e+00 - val_loss: 0.7150 - val_accuracy: 0.0000e+00
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.7106 - accuracy: 0.0000e+00 - val_loss: 0.7117 - val_accuracy: 0.0000e+00
Epoch 5/10
25/25 [==============================] - 0s 1ms/step - loss: 0.7072 - accuracy: 0.0000e+00 - val_loss: 0.7084 - val_accuracy: 0.0000e+00
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 0.7039 - accuracy: 0.0000e+00 - val_loss: 0.7051 - val_accuracy: 0.0000e+00
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 0.7006 - accuracy: 0.0000e+00 - val_loss: 0.